# Natural Language Processsing - SMS Spam Detection Filter

Aim - Use SMS data to build a Spam Detection model.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [4]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [5]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
# Check for any missing values
from pyspark.sql.functions import isnan, isnull, when, count, col

data.select([count(when(isnan(c)| isnull(c), c)).alias(c) for c in data.columns]).show()

+-----+----+
|class|text|
+-----+----+
|    0|   0|
+-----+----+



In [7]:
# Create a length column containing the text length

from pyspark.sql.functions import length

data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [8]:
# Calculate the mean ham and spam length
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



There is a significant difference in the mean lengths of ham and spam messages. 

To transform the features and labels in the format compatible with classification models we have to follow a few steps:
1. Convert the text into words (Tokenizer)
2. Remove the common words (StopWordsRemover)
3. Convert the corpus into vectors of word counts (CountVectorizer)
4. Get the importance of a words in the corpus (IDF)
5. Index label from String to Numeric value (StringIndexer)
6. Assemble the TF-IDF and text length vectors to form the feature vector (VectorAssembler)

In [9]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,CountVectorizer,
                                IDF,StringIndexer)

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
class_to_num = StringIndexer(inputCol='class',outputCol='label')

In [10]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

assembler = VectorAssembler(inputCols=['tf_idf','length'],
                            outputCol='features')

In [11]:
# Use Pipeline since there are a number of steps for data preparation

from pyspark.ml import Pipeline

data_prep_pipe = Pipeline(stages=[class_to_num,
                                  tokenizer,
                                  stopremove,
                                  count_vec,
                                  idf,
                                  assembler])

In [12]:
final_data = data_prep_pipe.fit(data).transform(data)

In [13]:
final_data = final_data.select(['label','features'])

In [14]:
final_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [15]:
# Split the data 7:3

train,test = final_data.randomSplit([0.7,0.3])

In [16]:
# We will use the classification model Naive Bayes

from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()

In [17]:
spam_detector = nb.fit(train)

In [18]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [19]:
final_data.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [20]:
results = spam_detector.transform(test)

In [21]:
results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,7,8,1...|[-878.27983738708...|[1.0,5.7898717117...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-656.72512798450...|[1.0,8.0914068539...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-541.20149158225...|[1.0,4.7600332174...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.93494669810...|[1.0,6.1699216451...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-687.28474760614...|[1.0,1.9573680761...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-687.38226278891...|[1.0,2.0189281044...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-806.80024024406...|[1.0,1.4070294473...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-851.88353728123...|[1.0,3.3487077097...|       0.0|
|  0.0|(13424,[0,1,31,43...|[-345.95784889121...|[1.0,4.3317831179...|       0.0|
|  0.0|(13424,[0

In [23]:
# Evaluate the results using a MulticlassClassificationEvaluator

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(results)
print("Accuracy of model at detecting spam was: {}".format(acc))

Accuracy of model at detecting spam was: 0.9192846798623656


Our model is 92% accurate at detecting spam SMS which is good enough!

-------------------------------------------------------